# Convolutional Neural Networks

Also called Convolutional Networks, **Convolutional Neural Networks** (CNN) are algorithms specialised in processing data that has a **known grid-like form**, typical examples include 2-D images and time series (where one dimension is given by the time-sequence or samples and the other by the features defining the sample).

The name comes from the mathematical operation that these algorithms use, known as **_"convolution"_**. In this sense, CNNs are Neural Networks that use convolution instead of general matrix multiplication in at least one of their layers.

### What is convolution?

In mathematics, convolution is an operation on two functions that produces a third function representing how the shape of one of these functions is modified by the effect of the other function (the term convolution refers bith to the resulting function and to the operation itself).

The convolution in mathematics is normally defined for continuous spaces as the operation '$*$' between function f and g as:

$$(f*g)(t) = \int_{-\infty}^{\infty} f(t+\tau)g(\tau)d\tau$$

Now, we said before that we will use convolution for grid-like inputs. Imagine then that we will apply the convolution function to an image X, sized (m,n) pixels. Since the input is now discrete, the expression would be:

$$ (X*cv)[i,j] = \sum_{i=0}^{m} \sum_{j=0}^{n}X[i+m, j+n]cv[m,n] $$

Where 'x' is a vector (also called **tensor**) with the sample input data and '*cv*' is the convolusional function, also called '**kernel**' or '**filter**'.

>There are two important things to note about the equations above, the first is that I'm using the commutative version of the convolution (in the standard version, $\tau$ is applied to the convolution function, not the input function). The other is that a substraction operation is normally performed instead of a sum, that is $f(t-\tau)$ and $X[i-m,j-n]$ are used in the standard version of the equations. The version above is known as '**cross-correlation** and it is what's used in most neural network libraries calling it 'convolution'.

The following image shows a convolution between a (3,4) **tensor** as input and a (2,2) kernel. Notice that the operation is not equivalent to a matrix multiplication, but an element-wise multiplication where we take a 'section' the size of the kernel and 'move it around' the tensor, one element at a time towards the right and down, to complete the output. 

![Convolution example](images/convolution_ex1.png "convolution example")

> We can see that the top-left element (location [1,1]) of the output would then be the element-wise multiplication of the elements in the kernel and those in the input on a block the same size of the kernel (*aw+bx+cy+fz*). The following element to the right (location [1,2]) would be the element-wise multiplication of the kernel and the elements in the tensor that result from moving the block once to the right (*bw+cx+fy+gz*).


Notice that, if we have an input/kernel with size (m,n) and a kernel size (f,f), then the size of the output $ X_{(m,n)} * K_{(f,f)} = Conv_{(c1, c2)} $ will have the form:

$$(c1, c2) = (m - f + 1, n - f + 1)$$

Now, let's look at the following example. We have an image size 6x6 pixels and 3 colour channels (Red, Green and Blue, RGB) and 2 filters or kernels to convolute it with. As we can see from the previous equation, the size of the output is (4,4) with one matrix per filter.


![Convolution example 2](images/convolution_ex2.png "convolution example 2")

The general equation to determine the number of parameters in a layer (assuming square-sized filters) is:

$$ Number\ of\ parameters = (filter\_size^2\ x\ input\_channels + 1)x\ number\_of\_filters $$

Notice that the '1' in the equation represents the bias component, which is 1 per filter. 

Thus, for the previous example, we have: 56 parameters = (3x3x3 + 1)x 2 = 56. Notice that if we had a Fully Connected (FC) regular NN layer sized (3,1), we would have (6x6x3x3 + 3) = 327 parameters (which comes from (input_sizes x input channels x layer's size + layer's bias).

## Padding and striding

There are 2 common operarations used in convolution that we need to mention. We can see from the 1st image above that the elements at the border of the input (edges) will usually be under-represented with respect to the internal elements, since the kernel 'passes' through the elements on the edges fewer times. To reduce this effect, we use **_padding_**, that's adding zeros to the borders. Padding is represented normally as 'p', and p=1 means we include one more dimension to the input.
 
Sometimes we might want to skip over some positions of the input to reduce computational cost (at the expense of reducing the 'resolution' of the features we extract). This operation is known as **_stride_** and consists on moving the kernel more than one element at a time. A stride of 2 (s=2) means that we move over the input on steps of 2 instead of 1.

Notice that padding and striding will change the size of the output, so the new equation including both effects will be: 

$$ (c1, c2) = ( \frac{m + 2p -f}{s} + 1, \frac{n + 2p -f}{s} + 1 )  $$

Where (m,n) represent the input size, 'p' is the number zeros we add at the border and 's' the number of elements we use to move along the input (s=1 means steps of 1 element, as we did before).

What are the benefits of this operation versus the regular matrix multiplication? We have at least 2 improvements with regards to the matrix multiplication of traditional Neural Networks:

- **Sparce interactions**: Traditional matrix multiplication in Neural Networks every input unit intereacts with every output unit. Since convolutional networks use *kernels with size much smaller* than that of the tensor, we have fewer interactions between intput and output units, reducing the number of parameters and memory requirements of the model. We can see this on the previous example, where the input '*b*' is present only in the first and second elements of the output.
- **Parameter sharing**: Traditional neural nets use each element of the weight matrix only once when computing the output of a layer. In a convolutional neural net, each element of the kernel is used at every position of the input (tensor). We can see on the example above that all elements of the kernel are present in every element of the output. This parameter sharing means that instead of learning a separate set of parameters for every location, we learn only one set per layer. This allows us to reduce the storage requirement of the model to just the number of elements in the kernel (which tends to be several orders of magnitude smaller than that of the input).  

![Connection example](images/NN_vs_CNN_connections.png "Connections example")
> This image shows the effects of **sparce interaction** and **parameter sharing**, using an input tensor X, an output S, and a (3,1) kernel. The dark black arrows show the output units that are connected to the input using the central element of the 3-element kernel. As we can see, every element of the output $S_i$ uses the central parameter of the kernel in the Convolutional Net model. In the traditional Neural Net, only $S_3$ in the output uses the central element of the kernel, demonstrating the efficiency gained with parameter sharing. 
The grey arrows show the sparce connectivity of the convolutional model. We see that all units in the output are connected to the input units in a regular neural net, while any input unit affects at most 3 units in the output in the convolutional model (notice also that the units at the border of the input -1st and last- have fewer connections than those between them... we'll review the effect of this later on).

(*Note: This example was extracted from the book "Deep Learning" by Goodfellow et al.*)

![Convolution stages](images/convolution_stages.png "Convolution stages")

A typical convolutional net consists of 3 stages:

- **1st stage**: produces a set of linear activations through several parallel convolutions, regularly called **convolution stage**.
- **2nd stage**: the result of the convolutions is passed through a non-linear activation function (ex: ReLU). This is often called **detector stage**
- **3rd stage**: a **pooling** function is applied to the output of the detector stage to 'summarise its responses over a neightbourhood', improving the computational efficiency since the next layer has fewer inputs to process. Common examples of pooling functions are **Max pooling** (that returns only the max value in a segment) and **Average pooling** (that returns the average value of a segment).

### Convolutional Neural Network Example

The following example was taken from the course **"Convolutional Neural Networks" by DeepLearning.ai** in Coursera. It shows a simple digit classification exercise using convolution, 2 regular full-connected (FC) neural layers, and a Softmax function to define the output between digits 0 and 9.

![Convolution network example](images/convolution_ex3_1.png "Convolution example 3")

We can also see the changes in parameters and activation size on the following table:

![Convolution network example](images/convolution_ex3_2.png "Convolution example 3")